# Course 10 Lab 4 - Exploratory Data Analysis

In [4]:
# import libraries
from dotenv import load_dotenv
import os
load_dotenv() # reads the env file in our repository
import ibm_db
import ibm_db_sa
import sqlalchemy
from sqlalchemy import *
%load_ext sql
import pandas as pd
import csv, sqlite3

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
# Connect to the database
dsn_hostname = os.environ.get("HOSTNAME")     
dsn_uid = os.environ.get("USERNAME")      
dsn_pwd = os.environ.get("PASSWORD")

%sql ibm_db_sa://{dsn_uid}:{dsn_pwd}@{dsn_hostname}:31249/bludb;security=SSL

In [5]:
con = sqlite3.connect('my_data1.db')
cur = con.cursor()

In [6]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

c:\Users\sspal\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [37]:
# display the names of the unique launch sites in the space mission
%sql select unique(LAUNCH_SITE)  from SPACEXTBL;

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


launch_site
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E


In [13]:
# Display 5 records where lanch sites begin with the string 'CCA'
%sql select * from SPACEXTBL where LAUNCH_SITE like 'CCA%' limit 5;

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


DATE,time__utc_,booster_version,launch_site,payload,payload_mass__kg_,orbit,customer,mission_outcome,landing__outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [14]:
# display the total payload mass carried by boosters launched by NASA (CRS)
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where "customer" = 'NASA (CRS)';

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


1
45596


In [16]:
# display the average payload mass carried by booster version F9 v1.1
%sql select avg(PAYLOAD_MASS__KG_) from SPACEXTBL where BOOSTER_VERSION = 'F9 v1.1';

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


1
2928


In [17]:
# list the date when the first successful landing outcome in ground pad was achieved
%sql select min(DATE) from SPACEXTBL where LANDING__OUTCOME = 'Success (ground pad)';

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


1
2015-12-22


In [18]:
# list the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql select BOOSTER_VERSION from SPACEXTBL where LANDING__OUTCOME = 'Success (drone ship)' and PAYLOAD_MASS__KG_ between 4000 and 6000;

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


booster_version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [24]:
# list the total number of successful and failure mission outcomes
%sql select MISSION_OUTCOME, count(MISSION_OUTCOME) from SPACEXTBL group by MISSION_OUTCOME;

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


mission_outcome,2
Failure (in flight),1
Success,99
Success (payload status unclear),1


In [25]:
# list the names of the booster_versions which have carried the maximum payload mass, use a subquery
%sql select BOOSTER_VERSION from SPACEXTBL where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTBL);

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


booster_version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [33]:
# List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.
# Note: SQLLite does not support monthnames. So you need to use substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.
%sql select MONTH(DATE), LANDING__OUTCOME, BOOSTER_VERSION, LAUNCH_SITE from SPACEXTBL where EXTRACT(YEAR FROM DATE) and LANDING__OUTCOME = 'Failure (drone ship)';

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


1,landing__outcome,booster_version,launch_site
1,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
4,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
1,Failure (drone ship),F9 v1.1 B1017,VAFB SLC-4E
3,Failure (drone ship),F9 FT B1020,CCAFS LC-40
6,Failure (drone ship),F9 FT B1024,CCAFS LC-40


In [35]:
# rank the count of successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order
%sql select LANDING__OUTCOME from SPACEXTBL where DATE between '2010-06-04' and '2017-03-20' order by DATE desc;

 * ibm_db_sa://myb97303:***@b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:31249/bludb;security=SSL
Done.


landing__outcome
No attempt
Success (ground pad)
Success (drone ship)
Success (drone ship)
Success (ground pad)
Failure (drone ship)
Success (drone ship)
Success (drone ship)
Success (drone ship)
Failure (drone ship)
